In [11]:
import numpy as np
import pandas as pd
import torch
from utils.utils import read_json_file
from source.data_ingest import load_dataset
from source.configuration import Configuration
from source.data_processing import process_data, cal_pu_gpm
from source.building_data_loader import create_data_loader
from source.model import create_model
from source.train_model import train_model
from source.evaluation import eval_experiment, inference_dl, Evaluation

### 1. Read Data and Configuration

In [12]:
#0. Read configuration
config = read_json_file("config.json")
config_class = Configuration(config)
T = config_class.get_T()

gpu_idx = config_class.get_gpu_index()
device = torch.device("cpu")
print(f"0. Using device: {device}")

batch_size = config_class.get_batch_size()
#1. Open dataset
ann_max_gpm, ann_max_stas, df_stasiun = load_dataset(config_class)
print("Loading Dataset DONE!")

0. Using device: cpu
Loading Dataset DONE!


Data yang digunakan:
1. ann_max_gpm: data annual maxima gpm yang berbentuk "Xarray Dataset"
2. ann_max_stas: data annual maxima pada setiap stasiun yang berbentuk "Dictionary". Nama Stasiun (Key) dan Nilai ann max CH (value)
3. df_stasiun: data detail tentang setiap titik stasiun berbentuk "Dataframe". Features nya adalah Nama Stasiun, Elevasi, Jarak, Lintang, dan Bujur 

### 2. Data Processing

In [13]:
#2. Processing Dataset
X_train, y_train, X_val, y_val = process_data(max_gpm=ann_max_gpm, 
                                              max_stas=ann_max_stas,
                                              df_stas=df_stasiun,
                                              config_class=config_class)
pu_gpm = cal_pu_gpm(X_val,T)
 
X_train = torch.tensor(X_train,dtype=torch.float32).to(device)
y_train = torch.tensor(y_train,dtype=torch.float32).to(device)
X_val = torch.tensor(X_val,dtype=torch.float32).to(device)
y_val = torch.tensor(y_val,dtype=torch.float32).to(device)

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

print("Processing Dataset DONE!")

torch.Size([81, 20, 3, 3]) torch.Size([81, 6, 1]) torch.Size([14, 20, 3, 3]) torch.Size([14, 6, 1])
Processing Dataset DONE!


### 3. Building Model and Data Loader

In [14]:
#3. Create Data Loader
train_loader = create_data_loader(X=X_train, y=y_train, batch_size=batch_size, shuffle= True)
val_loader = create_data_loader(X=X_val, y=y_val, batch_size=batch_size, shuffle= False)

#4. Create Model
model = create_model(config_class=config_class)

print("Creating Model.. DONE!")

Creating Model.. DONE!


In [15]:
print(model)

RegCNN(
  (conv1): Conv2d(20, 16, kernel_size=(2, 2), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=32, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=6, bias=True)
)


In [16]:
#5. Train Model
model, train_loss, eval_loss = train_model(config_class= config_class, model=model, train_loader=train_loader,
                                           X_val=X_val, y_val=y_val, device=device)
print("Training model.. DONE")
pu_predicted = inference_dl(model=model, X=X_val)

Epoch [1/30], Train Loss: 33192.1914, Eval Loss: 23267.291015625
Epoch [2/30], Train Loss: 10789.6162, Eval Loss: 2941.08251953125
Epoch [3/30], Train Loss: 1545.3115, Eval Loss: 2728.542236328125
Epoch [4/30], Train Loss: 4749.4165, Eval Loss: 2561.2275390625
Epoch [5/30], Train Loss: 742.5867, Eval Loss: 1127.8267822265625
Epoch [6/30], Train Loss: 1874.7125, Eval Loss: 1785.406982421875
Epoch [7/30], Train Loss: 1626.1996, Eval Loss: 1091.7708740234375
Epoch [8/30], Train Loss: 377.0281, Eval Loss: 994.7647094726562
Epoch [9/30], Train Loss: 58.5625, Eval Loss: 980.7899780273438
Epoch [10/30], Train Loss: 260.0649, Eval Loss: 1117.962158203125
Epoch [11/30], Train Loss: 2092.1831, Eval Loss: 984.4395751953125
Epoch [12/30], Train Loss: 1284.2982, Eval Loss: 1084.0966796875
Epoch [13/30], Train Loss: 200.3305, Eval Loss: 1082.0059814453125
Epoch [14/30], Train Loss: 864.9008, Eval Loss: 1086.5885009765625
Epoch [15/30], Train Loss: 1852.2081, Eval Loss: 1055.09912109375
Epoch [16/30]

In [17]:
y_val = y_val.view(len(y_val),-1)
y_val = y_val.detach().numpy()
y_val

array([[127.181656, 162.78029 , 186.34967 , 216.12967 , 238.22217 ,
        260.15152 ],
       [ 96.5341  , 123.85648 , 141.94627 , 164.80278 , 181.75903 ,
        198.59009 ],
       [100.82792 , 123.35089 , 138.26306 , 157.10461 , 171.08237 ,
        184.95691 ],
       [101.12875 , 128.47421 , 146.5793  , 169.45511 , 186.4257  ,
        203.27098 ],
       [101.924355, 125.23868 , 140.67479 , 160.17834 , 174.6472  ,
        189.00923 ],
       [ 98.81805 , 124.15775 , 140.93484 , 162.13274 , 177.85855 ,
        193.46825 ],
       [119.621864, 168.89409 , 201.51659 , 242.7352  , 273.31354 ,
        303.66608 ],
       [ 87.19409 , 102.45357 , 112.55667 , 125.32197 , 134.79199 ,
        144.1921  ],
       [127.4857  , 158.98409 , 179.83876 , 206.18869 , 225.73659 ,
        245.14012 ],
       [135.59276 , 173.40492 , 198.43987 , 230.07158 , 253.5378  ,
        276.83075 ],
       [ 90.204575, 123.099884, 144.87944 , 172.39796 , 192.81277 ,
        213.07686 ],
       [128.00989 , 1

In [18]:
pu_predicted = pu_predicted.detach().numpy()
pu_predicted

array([[109.00395 , 139.35217 , 158.37428 , 184.09036 , 205.47139 ,
        222.21768 ],
       [ 97.20059 , 125.219696, 142.5634  , 165.71599 , 183.02477 ,
        197.50354 ],
       [ 91.56268 , 116.36508 , 133.71312 , 155.03345 , 172.99788 ,
        185.65215 ],
       [110.53283 , 142.12984 , 162.74782 , 189.20967 , 209.1407  ,
        223.19131 ],
       [ 90.22627 , 115.9955  , 131.43886 , 154.24959 , 172.28865 ,
        183.89091 ],
       [114.831985, 152.49231 , 171.88086 , 204.0429  , 222.91533 ,
        242.19981 ],
       [118.03749 , 149.33217 , 167.02245 , 196.71751 , 216.21362 ,
        236.42319 ],
       [ 84.71616 , 109.92989 , 124.91218 , 144.27356 , 160.8817  ,
        172.25247 ],
       [112.80264 , 141.10596 , 159.4855  , 185.25204 , 207.26141 ,
        221.35689 ],
       [106.89395 , 138.88165 , 160.16501 , 179.38512 , 206.01161 ,
        217.17462 ],
       [ 99.94656 , 128.39821 , 143.99495 , 166.27429 , 187.36562 ,
        199.78589 ],
       [147.80414 , 1

### 6. Evaluation 

In [19]:
evaluation = Evaluation(model)
nmse = []
rmse = []
for i in range(6):
    nmse.append(evaluation.calculate_nmse_mean(pu_predicted[:,i],y_val[:,i]))
    rmse.append(evaluation.calculate_rmse(pu_predicted[:,i],y_val[:,i]))

In [20]:
df_evaluation = pd.DataFrame(
    {"Periode Ulang" : [2,5,10,25,50,100],
     "NMSE" : nmse,
     "RMSE" : rmse})
df_evaluation

,Periode Ulang,NMSE,RMSE
0,2,0.125406,13.981305
1,5,0.145420,21.146109
2,10,0.158044,26.531616
3,25,0.180945,35.511459
4,50,0.182874,39.740311
5,100,0.197726,47.099865


In [21]:
#df_evaluation.to_excel("RMSE dan NMSE GPM Correction CNN.xlsx")

In [22]:
#torch.save(model, 'Reg Model CNN.pth')

### 7. Visualization

In [ ]:
import matplotlib.pyplot as plt

# Ganti nmse_all dengan nmse_all_before atau nmse_difference sesuai kebutuhan
data_to_visualize = nmse_difference  # atau nmse_difference

# Extract data for visualization
periods_to_plot = [2, 5, 10, 25, 50, 100]

# Inisialisasi dictionary untuk menyimpan nilai NMSE untuk setiap periode ulang
nmse_by_period = {period: [] for period in periods_to_plot}

# Populate dictionary dengan nilai NMSE sesuai periode ulang
for result in data_to_visualize:
    period = result[0]
    if period in periods_to_plot:
        nmse_by_period[period].append(result[2])

# Buat boxplot
fig, ax = plt.subplots()
ax.boxplot(nmse_by_period.values(), labels=nmse_by_period.keys())#, showfliers=False)
ax.set_xlabel('Return Period (Yr)')
ax.set_ylabel('NMSE Mean')
ax.set_title('Difference NMSE Mean (Simulasi A)')

# Tampilkan plot
plt.show()

In [ ]:
# #6. Eval experiment
# mse_stas_gpm, mse_stas_predicted = eval_experiment(pu_gpm=pu_gpm, pu_predicted=pu_predicted, 
#                                                    pu_stas=y_val, config_class=config_class)

# print("Evaluating Experiment.. DONE!")